In [10]:
import pandas as pd
import numpy as np
import scipy.stats

import altair as alt
import sys
import ast
sys.path.append('/home/jwerner/BrainPower/brainpower/submodule/')
import bp_preprocessing

Import stats csvs. Convert ind_scores from str to list

In [11]:
sys.path.append('/home/jwerner/BrainPower/data/split_data')
ridge_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/ridge_10_30_features.csv')
ridge_stats['ind_scores'] = ridge_stats['ind_scores'].apply(ast.literal_eval)
ridge_stats['features'] = ridge_stats['features'].apply(ast.literal_eval)

rforest_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/rforest_10_30_features.csv')
rforest_stats['ind_scores'] = rforest_stats['ind_scores'].apply(ast.literal_eval)
rforest_stats['features'] = rforest_stats['features'].apply(ast.literal_eval)


nbayes_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/nbayes_10_30_features.csv')
nbayes_stats['ind_scores'] = nbayes_stats['ind_scores'].apply(ast.literal_eval)
nbayes_stats['features'] = nbayes_stats['features'].apply(ast.literal_eval)


xgboost_stats = pd.read_csv('/home/jwerner/BrainPower/brainpower/data/split_data/xgboost100_features.csv')
xgboost_stats['ind_scores'] = xgboost_stats['ind_scores'].apply(ast.literal_eval)
xgboost_stats['features'] = xgboost_stats['features'].apply(ast.literal_eval)


In [16]:
def get_feature_dict(frame,uniqueness):
    count_dict = {}
    for i in range(0,len(ridge_stats)):
        # Convert the string into a list of lists
        list_of_lists = ridge_stats['features'].iloc[i]
        if uniqueness == "unique":
            lst = np.unique(list_of_lists)
            #print('features in unique list from feature_num',n,':',len(lst))
        elif uniqueness == "duplicate":
            lst, count = np.unique(list_of_lists, return_counts=True)
            lst = lst[count > 1]
            #print('features in duplicate list from feature_num',n,':',len(lst)) 
        count_dict[ridge_stats['feature_num'].iloc[i]] = [len(lst),lst]
    return count_dict

In [17]:
def features_ina_string(frame, uniqueness='unique'):    
    def unique_features_to_string(X,uniqueness):
        if type(X) == str:
            string = X
        elif type(X) == list:
            string = str(X)
        # remove the outer quotation marks and brackets
        string = string[2:-2]

        # split the string into a list of lists of values
        lst = [s.strip().split(", ") for s in string.split("], [")]
        lst = np.array(lst).flatten()

        if uniqueness == "unique":
            lst = np.unique(lst)
            #print('features in unique list from feature_num',n,':',len(lst))
        elif uniqueness == "duplicate":
            lst, count = np.unique(lst, return_counts=True)
            lst = lst[count > 1]
            #print('features in duplicate list from feature_num',n,':',len(lst))            

        else:
            return "Invalid value for parameter 'uniqueness'. Use 'unique' or 'duplicate'."
        hyphen_items = []
        for i, item in enumerate(lst):
            # if it's an odd index, add a hyphen before the item
            if i != 1 and i % 20 == 1:
                hyphen_items.append('- ' + item)
            else:
                hyphen_items.append(item)
        stringy = ", ".join(hyphen_items)
        stringy = stringy.replace("'", "")
        return stringy
    
    features_string = []
    count_dict = {}
    for i in range(0,len(frame)):
        feature = unique_features_to_string(frame['features'].iloc[i],uniqueness=uniqueness)
        features_string.append(feature)

    return features_string

In [20]:
def altair_feature_selection_chart(frame, uniqueness):
    """
    Assumes independent scores list is in ind_scores
    Assumes feature number is in feature_num

    """
    df = frame

    # features_ina_string assumes features are in 'features' column
    df["uniqueX_features_ina_string"] = features_ina_string(df, uniqueness=uniqueness)
    # def feature_selection_vis(frame,quantitative,ordinal)
    selection = alt.selection_single(empty="none")

    # Use the explode method to expand the dataframe for the error bars calculation
    df_exploded = df.explode("ind_scores").reset_index(drop=True)
    df_exploded.head(10)
    error_bars = (
        alt.Chart(df_exploded)
        .mark_boxplot()
        .encode(
            y=alt.Y("ind_scores:Q"),
            x=alt.X("feature_num:O"),
        )
        .add_selection(selection)
    )

    points = (
        alt.Chart(df)
        .mark_point(filled=True, color="black")
        .encode(
            y=alt.Y("avg_score:Q", title="score"),
            x=alt.X("feature_num:O", title="feature_num"),
        )
        .add_selection(selection)
    )

    # Create a list to split into multiple lines in the chart
    df["featuresX"] = (
        df["uniqueX_features_ina_string"].astype(str).str.split("-")
    )  # Create a list to split into multiple lines in the chart
    df["zero"] = 0
    text = (
        alt.Chart(df)
        .mark_text(baseline="bottom", dy=-75)
        .encode(y="zero", text="featuresX")
        .transform_filter(selection)
    )

    chart = (
        (error_bars + points + text)
        .properties(title="plot", height=600, width=1000)
        .interactive()
    )

    return chart

In [21]:
altair_feature_selection_chart(ridge_stats,uniqueness=3)

alt.LayerChart(...)

In [11]:
ridge_stats.drop(columns=['duplicate_features_string','unique_features_string','featuresX','zero'])

,feature_num,ind_scores,avg_score,stdev,ci68,yerr,features,duplicate_features_string,unique_features_string,featuresX,zero
0,10,"[0.3958333333333333, 0.37083333333333335, 0.32...",0.381667,0.052513,"(0.32944447525005116, 0.4338888580832822)",0.052222,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433G, 1433Z, AK1C1, CADH5, CAH2, CHLE, CLMP, ...","1433G, 1433Z, A0A075B7D0, AK1C1, AMY2B, AMYP, ...","[1433G, 1433Z, A0A075B7D0, AK1C1, AMY2B, AMYP,...",0
1,11,"[0.3083333333333333, 0.31666666666666665, 0.27...",0.359583,0.054552,"(0.30533414676906756, 0.41383251989759906)",0.054249,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2, CHLE,...","1433B, 1433G, 1433Z, A0A075B7D0, AK1C1, AMY2B,...","[1433B, 1433G, 1433Z, A0A075B7D0, AK1C1, AMY2B...",0
2,12,"[0.425, 0.3041666666666667, 0.2916666666666666...",0.382500,0.061061,"(0.3217771913902592, 0.44322280860974084)",0.060723,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2, CHLE,...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
3,13,"[0.3958333333333333, 0.3083333333333333, 0.329...",0.386250,0.063165,"(0.3234355423317144, 0.44906445766828557)",0.062814,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
4,14,"[0.3958333333333333, 0.3083333333333333, 0.287...",0.392083,0.080503,"(0.3120265140935771, 0.4721401525730896)",0.080057,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
5,15,"[0.38333333333333336, 0.29583333333333334, 0.2...",0.402500,0.092762,"(0.3102517093955644, 0.4947482906044357)",0.092248,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
6,16,"[0.38333333333333336, 0.3583333333333333, 0.37...",0.408333,0.073077,"(0.33566113150589033, 0.4810055351607764)",0.072672,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
7,17,"[0.38333333333333336, 0.3458333333333333, 0.37...",0.402917,0.064980,"(0.338296828359979, 0.4675365049733544)",0.064620,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1,...","[1433B, 1433G, 1433Z, A0A075B7D0, ADA28, AK1C1...",0
8,18,"[0.4208333333333333, 0.3458333333333333, 0.379...",0.421250,0.080287,"(0.34140793159435784, 0.5010920684056421)",0.079842,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433Z, AK1C1, AMY2B, CADH5, CAH2...","1433B, 1433G, 1433T, 1433Z, A0A075B7D0, ADA28,...","[1433B, 1433G, 1433T, 1433Z, A0A075B7D0, ADA28...",0
9,19,"[0.475, 0.3208333333333333, 0.4291666666666667...",0.420000,0.089264,"(0.33123074571740463, 0.5087692542825955)",0.088769,"[[TAU, POTEJ, MYDGF, 1433G, PVR, LTBP4, PAL4A,...","1433B, 1433G, 1433T, 1433Z, AK1C1, AMY2B, CADH...","1433B, 1433G, 1433T, 1433Z, A0A075B7D0, A1ATR,...","[1433B, 1433G, 1433T, 1433Z, A0A075B7D0, A1ATR...",0


In [12]:
alt.display.here()

TypeError: 'str' object is not callable

In [10]:
import altair as alt
import pandas as pd
from urllib.parse import urlencode

def make_google_query(name):
    return "https://www.google.com/search?" + urlencode({'q': '"{0}"'.format(name)})


data = [
    ["GO:0005874","microtubule", 0.590923058896654, -4.00372136407618, 3.93380303408685, 5.07433362293908, 0.743307611944267, 0],
    ["GO:0042555","MCM complex",0.0516392267501353,5.29409032883786,-0.438484234906433,4.01582063426207,0.736867388621876,0],
    ["GO:0005886","plasma membrane",15.5064680247866,-2.59930578712986,-4.79438349762051,6.49331205332051,0.980465972776413,4.124E-05],
    ["GO:0030173","integral component of Golgi membrane",0.0482779463204013,-0.0820996416106789,6.61844221537962,3.98659260682221,0.720783016873817,0.16417986],
    ["GO:0031083","BLOC-1 complex",0.0157955281823943,6.03044083325888,2.61728943021364,3.50147007210041,0.638408624494431,0.22740185],
    ["GO:0030532","small nuclear ribonucleoprotein complex",0.138166054523554,2.1939043417736,2.03060434260059,4.44321603416583,0.571526896999077,0.2622474],
    ["GO:0008250","oligosaccharyltransferase complex",0.0394539627330108,2.22238070210506,4.52148800747906,3.89894446686651,0.602647357590838,0.39260902]
]

columns = ["term_ID","description","frequency","plot_X","plot_Y","log_size","uniqueness","dispensability"]

df = pd.DataFrame(data, columns=columns)
df['url'] = df['term_ID'].apply(make_google_query)

selection = alt.selection_single(empty='none')

base = alt.Chart(df).mark_point(filled=True, fillOpacity=0.5).encode(
    y=alt.Y('plot_Y', title="Semantic Space Y"),
    x=alt.X('plot_X', title="Semantic Space X"),
    size=alt.Size('log_size', scale=alt.Scale(base=0, domain=[3.5,6], range=[1000, 7000])),
    color=alt.Color('uniqueness', scale=alt.Scale(scheme='viridis')),
    

    tooltip=[
             alt.Tooltip("description"), 
             alt.Tooltip("frequency"),
             alt.Tooltip("log_size"), 
             alt.Tooltip("uniqueness"),
             alt.Tooltip("dispensability")]
).add_selection(
    selection
)
 

df['long_description'] = ridge_stats['features'].astype(str).str.split(']') # Create a list to split into multiple lines in the chart

text = alt.Chart(df).mark_text(
).encode(
    y=alt.Y('plot_Y', title="Semantic Space Y"),
    x=alt.X('plot_X', title="Semantic Space X"),
    color=alt.Color('uniqueness', scale=alt.Scale(scheme='viridis')),
    text="long_description"
).transform_filter(
    selection
)

(base + text ).properties(
    title="plot",
    height=600,
    width=800
).interactive()

alt.LayerChart(...)

In [7]:
data = [
    ["GO:0005874","microtubule", 0.590923058896654, -4.00372136407618, 3.93380303408685, 5.07433362293908, 0.743307611944267, 0],
    ["GO:0042555","MCM complex",0.0516392267501353,5.29409032883786,-0.438484234906433,4.01582063426207,0.736867388621876,0],
    ["GO:0005886","plasma membrane",15.5064680247866,-2.59930578712986,-4.79438349762051,6.49331205332051,0.980465972776413,4.124E-05],
    ["GO:0030173","integral component of Golgi membrane",0.0482779463204013,-0.0820996416106789,6.61844221537962,3.98659260682221,0.720783016873817,0.16417986],
    ["GO:0031083","BLOC-1 complex",0.0157955281823943,6.03044083325888,2.61728943021364,3.50147007210041,0.638408624494431,0.22740185],
    ["GO:0030532","small nuclear ribonucleoprotein complex",0.138166054523554,2.1939043417736,2.03060434260059,4.44321603416583,0.571526896999077,0.2622474],
    ["GO:0008250","oligosaccharyltransferase complex",0.0394539627330108,2.22238070210506,4.52148800747906,3.89894446686651,0.602647357590838,0.39260902]
]

columns = ["term_ID","description","frequency","plot_X","plot_Y","log_size","uniqueness","dispensability"]

df = pd.DataFrame(data, columns=columns)

In [9]:
df['long_description'] = ridge_stats['features'].astype(str).str.split(']') # Create a list to split into multiple lines in the chart

selection = alt.selection_single(empty='none')

text = alt.Chart(df).mark_text(
).encode(
    y=alt.Y('plot_Y', title="Semantic Space Y"),
    x=alt.X('plot_X', title="Semantic Space X"),
    color=alt.Color('uniqueness', scale=alt.Scale(scheme='viridis')),
    text="long_description"
).transform_filter(
    selection
)
text

alt.Chart(...)

In [120]:
charts[1]

alt.LayerChart(...)

In [99]:
def feature_vis(frame,chart_title):
    import altair as alt
    import pandas as pd
    import numpy as np

    # generate some data points with uncertainties
    np.random.seed(0)
    x = frame['feature_num']
    y = frame['avg_score']
    yerr = frame['yerr']

    # set up data frame
    source = pd.DataFrame({"feature_num": x, "avg_score": y, "yerr": yerr})

    # the base chart
    base = alt.Chart(source)

    # generate the points
    points = base.mark_point(
        filled=True,
        size=50,
        color='black'
    ).encode(
        x=alt.X('feature_num', title='feature_num',scale=alt.Scale(domain=(9, 31))),
        y=alt.Y('avg_score', title='score',scale=alt.Scale(zero=False))
    ).properties(title=chart_title)

    # generate the error bars
    errorbars = base.mark_errorbar().encode(
        x="feature_num",
        y='yerr'
    )

    return points + errorbars

In [100]:
feature_vis(ridge_stats,'Ridge Classifier')

alt.LayerChart(...)